### Comparison methods for computing statistics within a polygon
Showing two methods on how statistics (in this case the mean) of all cells of a raster file within a polygon can be computed. 

One is rasterstats.zonal_stats which seems the status quo but is slow for large files
The other being to clip the raster to the polygon and then take the mean

This example includes all the data downloading

#### Set values and import libraries

In [1]:
from rasterstats import zonal_stats
import rioxarray
import geopandas as gpd
import pandas as pd
import numpy as np
import time
from pathlib import Path

In [14]:
data_dir="data"
hdx_url_name="malawi-administrative-level-0-3-boundaries"
hdx_dataset_name='mwi_adm_nso_20181016_SHP.zip'
hdx_dataset_dir=Path(data_dir) / 'mwi_adm_nso_20181016_SHP'
hdx_adm1_filename='mwi_admbnda_adm1_nso_20181016.shp'
hdx_adm1_path=hdx_dataset_dir / hdx_adm1_filename
chirps_path=Path(data_dir)/"chirps"/"chirps_daily_2020_p25.nc"

In [8]:
import shutil
import zipfile
from urllib.request import urlretrieve


In [ ]:
from hdx.hdx_configuration import Configuration 
from hdx.data.dataset import Dataset
Configuration.create(hdx_site='prod', user_agent='A_Quick_Example', hdx_read_only=True)

#### Download data

In [10]:
def hdx_download_filename(hdx_address,filename,directory):
    dataset = Dataset.read_from_hdx(hdx_address)
    resources = dataset.get_resources()
    for resource in resources:
        if resource["name"]==filename:
            _, path = resource.download()
            shutil.move(path, Path(directory) / filename)

In [15]:
hdx_download_filename(hdx_url_name,hdx_dataset_name,data_dir)

In [16]:
with zipfile.ZipFile(Path(data_dir)/hdx_dataset_name,"r") as zip_ref:
    zip_ref.extractall(hdx_dataset_dir)

In [17]:
#download chirps data
urlretrieve("https://data.chc.ucsb.edu/products/CHIRPS-2.0/global_daily/netcdf/p25/chirps-v2.0.2020.days_p25.nc", filename=chirps_path)

(PosixPath('../../../../Experiments/drought/Data/chirps/chirps_daily_2020_p25.nc'),
 <http.client.HTTPMessage at 0x136c4f1c0>)

#### Example for one date 
Simplest example, placed on Stack Overflow

In [18]:
ds=rioxarray.open_rasterio(chirps_path,masked=True)
ds=ds.rio.write_crs("EPSG:4326")

In [19]:
ds_date=ds.sel(time="2020-01-01").squeeze()

##### Compute with raster stats

In [20]:
start_time = time.time()
gdf = gpd.read_file(hdx_adm1_path)[["ADM1_EN", "geometry"]]
gdf["mean_adm"] = pd.DataFrame(
            zonal_stats(
                vectors=gdf,
                raster=ds_date.values,
                affine=ds_date.rio.transform(),
                nodata=np.nan,
                all_touched=False
            )
        )["mean"]
print("--- Raster stats ---")
display(gdf[["ADM1_EN","mean_adm"]])
print(f"--- Raster stats: {(time.time() - start_time):.2f} seconds ---")

--- Raster stats ---


,ADM1_EN,mean_adm
0,Central,14.624690
1,Northern,3.950312
2,Southern,20.649534


--- Raster stats: 15.43 seconds ---


##### Compute with clipping

In [21]:
start_time = time.time()
gdf = gpd.read_file(hdx_adm1_path)[["ADM1_EN", "geometry"]]
df_adm=pd.DataFrame(index=gdf.ADM1_EN.unique())
for a in gdf.ADM1_EN.unique():
    gdf_adm=gdf[gdf.ADM1_EN==a]

    da_clip = ds_date.rio.set_spatial_dims(
        x_dim="x", y_dim="y"
    ).rio.clip(
        gdf_adm["geometry"], all_touched=False
    )

    grid_mean = da_clip.mean(dim=["x", "y"],skipna=True).rename("mean_adm")
    df_adm.loc[a,"mean_adm"]=grid_mean.values
print("--- Rio clip ---")
display(df_adm)
print(f"--- Rio clip: {(time.time() - start_time):.2f} seconds ---")

--- Rio clip ---


,mean_adm
Central,14.624689
Northern,3.950313
Southern,20.649534


--- Rio clip: 0.29 seconds ---


#### Example for computation across several dates

In [22]:
#only select two dates for demonstration
ds_sel=ds.sel(time=slice("2020-01-01","2020-01-02"))

In [23]:
ds_sel

<xarray.DataArray 'precip' (time: 2, y: 400, x: 1440)>
[1152000 values with dtype=float32]
Coordinates:
  * time         (time) object 2020-01-01 00:00:00 2020-01-02 00:00:00
  * x            (x) float64 -179.9 -179.6 -179.4 -179.1 ... 179.4 179.6 179.9
  * y            (y) float64 49.88 49.62 49.38 49.12 ... -49.38 -49.62 -49.88
    spatial_ref  int64 0
Attributes:
    geostatial_lat_max:  50
    geostatial_lat_min:  -50
    geostatial_lon_max:  180
    geostatial_lon_min:  -180
    long_name:           Climate Hazards group InfraRed Precipitation with St...
    standard_name:       convective precipitation rate
    time_step:           day
    units:               ('mm/day', 'mm/day', 'mm/day', 'mm/day', 'mm/day', '...
    scale_factor:        1.0
    add_offset:          0.0

In [24]:
def compute_zonal_stats_rs(
    ds,
    adm_path,
    time_col="time",
    all_touched=False,
):
    """use zonal_stats to compute statistics on level in adm_path for all dates in ds"""
    df_list = []
    for t in ds[time_col].values:
        gdf = gpd.read_file(adm_path)[["ADM1_EN", "geometry"]]
        ds_date = ds.sel(time=t)

        gdf["mean_cell"] = pd.DataFrame(
            zonal_stats(
                vectors=gdf,
                raster=ds_date.values,
                affine=ds_date.rio.transform(),
                nodata=np.nan,
                all_touched=all_touched
            )
        )["mean"]

        gdf["time"] = pd.to_datetime(t.strftime('%Y-%m-%d'))

        df_list.append(gdf)
    df_zonal_stats = pd.concat(df_list)
    df_zonal_stats = df_zonal_stats.drop("geometry", axis=1)

    return df_zonal_stats

In [25]:
start_time = time.time()
df_stats_rs=compute_zonal_stats_rs(ds_sel,hdx_adm1_path)
print(f"--- Raster stats: {(time.time() - start_time):.2f} seconds ---")
df_stats_rs

--- Raster stats: 17.36 seconds ---


,ADM1_EN,mean_cell,time
0,Central,14.624690,2020-01-01
1,Northern,3.950312,2020-01-01
2,Southern,20.649534,2020-01-01
0,Central,17.869215,2020-01-02
1,Northern,11.116490,2020-01-02
2,Southern,18.518767,2020-01-02


In [26]:
def compute_zonal_stats_clip(
    da, 
    adm_path, 
    lon_coord="x",
    lat_coord="y",
    all_touched=False,
):
    """use rio.clip to compute statistics on level in adm_path for all dates in ds"""
    df_list=[]
    gdf = gpd.read_file(adm_path)[["ADM1_EN", "geometry"]]
    for a in gdf.ADM1_EN.unique():
        gdf_adm=gdf[gdf.ADM1_EN==a]
    
        da_clip = da.rio.set_spatial_dims(
            x_dim=lon_coord, y_dim=lat_coord
        ).rio.clip(
            gdf_adm["geometry"], all_touched=all_touched
        )

        grid_mean = da_clip.mean(dim=[lon_coord, lat_coord],skipna=True).rename("mean_adm")
        df_adm=grid_mean.to_dataframe().reset_index().drop("spatial_ref",axis=1)
        df_adm["ADM1_EN"]=a
        df_list.append(df_adm)

    df_zonal_stats = pd.concat(df_list)
    df_zonal_stats.time=pd.to_datetime(df_zonal_stats.time.apply(lambda x: x.strftime('%Y-%m-%d')))
    return df_zonal_stats

In [27]:
start_time = time.time()
df_stats_clip=compute_zonal_stats_clip(ds_sel.rio.write_crs("EPSG:4326"),hdx_adm1_path)
print(f"--- Clip and mean: {(time.time() - start_time):.2f} seconds ---")
df_stats_clip

--- Clip and mean: 0.37 seconds ---


,time,mean_adm,ADM1_EN
0,2020-01-01,14.624689,Central
1,2020-01-02,17.869215,Central
0,2020-01-01,3.950313,Northern
1,2020-01-02,11.116489,Northern
0,2020-01-01,20.649534,Southern
1,2020-01-02,18.518766,Southern
